In [1]:
!pip install -q --upgrade transformers datasets evaluate scikit-learn torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from datasets import Dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
import evaluate

In [3]:
df = pd.read_csv("/content/reply_classification_dataset.csv")

In [4]:
df.head()

,reply,label
0,Can we discuss pricing??,NEUTRAL
1,"Im excited to explore this further, plz send c...",POSITIVE
2,We not looking for new solutions.,negative
3,Could u clarify features included?,neutral
4,"lets,, schedule a meeting to dive deeper",positive


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   reply   2129 non-null   object
 1   label   2129 non-null   object
dtypes: object(2)
memory usage: 33.4+ KB


In [11]:
df["label"] = df["label"].str.strip().str.lower()

In [10]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^a-z\s]", '', text)
    return text.strip()

In [12]:
df["reply"] = df["reply"].apply(clean_text)
df = df.dropna()

In [13]:
print("Unique labels:", df["label"].unique())

Unique labels: ['neutral' 'positive' 'negative']


In [14]:
print(df["label"].value_counts())

label
positive    710
negative    710
neutral     709
Name: count, dtype: int64


In [15]:
df = df[df["label"].map(df["label"].value_counts()) > 1]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df["reply"], df["label"], test_size=0.2, random_state=42, stratify=df["label"])

In [17]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [18]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_test_tfidf)

In [19]:
print("\n=== Logistic Regression Baseline ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr, average="weighted"))
print(classification_report(y_test, y_pred_lr))


=== Logistic Regression Baseline ===
Accuracy: 1.0
F1 Score: 1.0
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       142
     neutral       1.00      1.00      1.00       142
    positive       1.00      1.00      1.00       142

    accuracy                           1.00       426
   macro avg       1.00      1.00      1.00       426
weighted avg       1.00      1.00      1.00       426



In [20]:
unique_labels = sorted(df["label"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

In [21]:
print("label2id:", label2id)


label2id: {'negative': 0, 'neutral': 1, 'positive': 2}


In [22]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df["reply"], df["label"], test_size=0.2, random_state=42, stratify=df["label"])

In [23]:
train_data = Dataset.from_dict({"text": train_texts, "label": [label2id[l] for l in train_labels]})
test_data = Dataset.from_dict({"text": test_texts, "label": [label2id[l] for l in test_labels]})

In [24]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [25]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)


In [26]:
train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

Map:   0%|          | 0/1703 [00:00<?, ? examples/s]

Map:   0%|          | 0/426 [00:00<?, ? examples/s]

In [27]:
train_data = train_data.remove_columns(["text"])
test_data = test_data.remove_columns(["text"])


In [28]:
train_data.set_format("torch")
test_data.set_format("torch")

In [29]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [31]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    }

In [32]:
from transformers import TrainingArguments, Trainer

In [33]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs"
)


In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3937505755.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivasanthosh2255 (shivasanthosh2255-excelr-solutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=214, training_loss=0.18291232741881755, metrics={'train_runtime': 2415.7919, 'train_samples_per_second': 1.41, 'train_steps_per_second': 0.089, 'total_flos': 112798001511936.0, 'train_loss': 0.18291232741881755, 'epoch': 2.0})

In [52]:
acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, average="weighted")

In [54]:
acc_bert = metrics["eval_accuracy"]
f1_bert = metrics["eval_f1"]

In [55]:
print("\n=== Model Comparison ===")
print(f"Logistic Regression - Accuracy: {acc_lr:.4f}, F1: {f1_lr:.4f}")
print(f"DistilBERT - Accuracy: {acc_bert:.4f}, F1: {f1_bert:.4f}")

if f1_bert > f1_lr:
    print("\n✅ Recommendation: DistilBERT for production (better generalization).")
else:
    print("\n✅ Recommendation: Logistic Regression (lighter, faster, competitive performance).")


=== Model Comparison ===
Logistic Regression - Accuracy: 1.0000, F1: 1.0000
DistilBERT - Accuracy: 1.0000, F1: 1.0000

✅ Recommendation: Logistic Regression (lighter, faster, competitive performance).


In [36]:
metrics = trainer.evaluate()
print("\n=== DistilBERT Fine-tuned ===")
print(metrics)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



=== DistilBERT Fine-tuned ===
{'eval_loss': 0.009859610348939896, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 85.9594, 'eval_samples_per_second': 4.956, 'eval_steps_per_second': 0.628, 'epoch': 2.0}


In [37]:
trainer.save_model("./results/best_model")


In [41]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
app =FastAPI()
class InputText(BaseModel):
    text: str
@app.post("/predict")
def predict(input: InputText):
    inputs = tokenizer(input.text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        conf, pred = torch.max(probs, dim=1)
    label = model.config.id2label[pred.item()]
    return {"label": label, "confidence": round(conf.item(), 2)}

Overwriting app.py


In [42]:
%%writefile answers.md
# Short Answers

**Q1: If you only had 200 labeled replies, how would you improve the model without collecting thousands more?**
I would use transfer learning with pretrained transformers, apply data augmentation (synonym replacement, back translation), and semi-supervised learning to leverage unlabeled data.

**Q2: How would you ensure your reply classifier doesn’t produce biased or unsafe outputs in production?**
I would ensure balanced training data, add bias checks, and deploy monitoring with human-in-the-loop review for edge cases.

**Q3: Suppose you want to generate personalized cold email openers using an LLM. What prompt design strategies would you use to keep outputs relevant and non-generic?**
I would use role-based instructions, provide few-shot examples, and constrain outputs with personalization tokens like name, role, and company.


Writing answers.md


In [51]:
%%writefile README.md
# Reply Classification Project

## Setup
```bash
pip install -r requirements.txt
import python libraries
upload datasets
Normalize labels
Clean text
Drop classes with < 2 samples (stratify needs ≥2 per class)
Train/test split
Baseline Model (Logistic Regression with TF-IDF)
Transformer Model (DistilBERT Fine-tuning)
Label mapping
Hugging Face dataset
model
metrics
training args
use %%writefile to create app.py and answers.md and README.md and create FastAPI use



Overwriting README.md
